# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [29]:
#Passo 1 - entrar na internet
#chrome ->chromedriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()

#Passo 2 - Pegar cotação do dólar
#entrar no google
navegador.get("https://www.google.com")

#pesquisar "cotação dólar"
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação dólar")

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

#pegar resultado que aparece no google
cotacao_dolar = navegador.find_element_by_xpath ('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)

#Passo 3 - Pegar cotação do Euro

#entrar no google
navegador.get("https://www.google.com")

#pesquisar "cotação euro"
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação euro")

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

#pegar resultado que aparece no google
cotacao_euro = navegador.find_element_by_xpath ('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

#Passo 4 - Pegar cotação do ouro

#entrar no sie do melhor cambio
navegador.get("https://www.melhorcambio.com/ouro-hoje")
#pegar a cotação do ouro
cotacao_ouro = navegador.find_element_by_xpath ('//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()


5.249
6.1929014250000005
300.62


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [30]:
#Passo 5 - importar e atualziar a bsase de dados

import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [31]:
#atualizar a cotação
#onde a coluna "moeda" = Dólar
#tabel.loc[linha,coluna]
tabela.loc[tabela ["Moeda"] == "Dólar","Cotação"] =float(cotacao_dolar)

tabela.loc[tabela ["Moeda"] == "Euro","Cotação"] =float(cotacao_euro)

tabela.loc[tabela ["Moeda"] == "Ouro","Cotação"] =float(cotacao_ouro)

#atualizar o preço de compra -> Preço Original * cotação

tabela["Preço Base Reais"] = tabela["Preço Base Original"] * tabela["Cotação"]

#atualizar o preço de venda -> Preço de compra * margem
tabela ["Preço Final"] = tabela["Preço Base Reais"] * tabela["Margem"]

#formatação

tabela ["Preço Final"] = tabela["Preço Final"].map("{:.2f}".format)
tabela ["Cotação"] = tabela["Cotação"].map("{:.2f}".format)
tabela ["Preço Base Reais"] = tabela["Preço Base Reais"].map("{:.2f}".format)

display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.25,5248.95,1.40,7348.53
1,Carro Renault,4500.00,Euro,6.19,27868.06,2.00,55736.11
2,Notebook Dell,899.99,Dólar,5.25,4724.05,1.70,8030.88
3,IPhone,799.00,Dólar,5.25,4193.95,1.70,7129.72
4,Carro Fiat,3000.00,Euro,6.19,18578.70,1.90,35299.54
5,Celular Xiaomi,480.48,Dólar,5.25,2522.04,2.00,5044.08
6,Joia 20g,20.00,Ouro,300.62,6012.40,1.15,6914.26


### Agora vamos exportar a nova base de preços atualizada

In [32]:
#Passo 6 - Exportar a base de dados atualizada

tabela.to_excel("ProdutosNovo.xlsx", index=False) #index =false -> para tirar numeração 